<a href="https://colab.research.google.com/github/aksh98/Collaborative_Filtering/blob/master/CFAssignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My Drive/Colab Notebooks
%cd Assignments_sem7

/content/drive/My Drive/Colab Notebooks
/content/drive/My Drive/Colab Notebooks/Assignments_sem7


In [0]:
import numpy as np
import pandas as pd
import json
import os
import sys
import pickle
import csv
import random
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from numpy import mean

from scipy.sparse.linalg import svds
# from surprise import Reader, Dataset, SVD, evaluate
from sklearn.metrics import mean_absolute_error
import math 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

print("Done")

Done


## Read data

In [0]:
file = 'FlickscoreData-26oct2018/ratings_correctFormat.json'
with open(file, encoding='latin1') as data_file:
    data = json.loads(data_file.read())
print(data)

[{'_id': '11megha89', 'rated': {'tt0104561': ['1'], 'tt0323013': ['1'], 'tt2213054': ['0'], 'tt1447508': ['0'], 'tt4505006': ['0'], 'tt0248126': ['1'], 'tt0420332': ['1'], 'tt0284137': ['1'], 'tt0296574': ['-1'], 'tt0367110': ['1'], 'tt2960140': ['0'], 'tt2356180': ['1'], 'tt0306434': ['1'], 'tt0088986': ['0'], 'tt3863552': ['0'], 'tt0116763': ['0'], 'tt1324059': ['1'], 'submit': ['submit'], 'tt3848892': ['1'], 'tt0107166': ['1'], 'tt4900716': ['1'], 'tt5286444': ['1'], 'tt4088588': ['0'], 'tt0110076': ['1'], 'tt1188996': ['1'], 'tt0299108': ['1']}}, {'_id': '9953547227 ', 'rated': {'tt0060689': ['0'], 'tt0375066': ['0'], 'tt0887769': ['0'], 'tt0106333': ['1'], 'tt0056806': ['0'], 'tt0085776': ['0'], 'tt1904875': ['0'], 'tt0316407': ['0'], 'tt0328810': ['0'], 'tt0306434': ['0'], 'tt3261022': ['1'], 'tt0066758': ['0'], 'tt0283005': ['0'], 'tt0072783': ['0'], 'tt0833533': ['1'], 'tt0459293': ['0'], 'submit': ['submitexit'], 'tt2187153': ['0'], 'tt0415144': ['0'], 'tt1620933': ['-1'], 'tt

In [0]:
file = 'FlickscoreData-26oct2018/ratings_correctFormat.json'
with open(file) as train_file:
    dict_train = json.loads(train_file.read())

# converting json dataset from dictionary to dataframe
train = pd.DataFrame(dict_train)
train.reset_index(level=0, inplace=True)
print(train.shape)
train.head()

(763, 3)


,index,_id,rated
0,0,11megha89,"{'tt0104561': ['1'], 'tt0323013': ['1'], 'tt22..."
1,1,9953547227,"{'tt0060689': ['0'], 'tt0375066': ['0'], 'tt08..."
2,2,ABCDEFGHI JKLM,"{'tt0079221': ['1'], 'tt5121000': ['1'], 'tt49..."
3,3,ANAND,"{'tt0070498': ['0'], 'tt0048987': ['0'], 'tt00..."
4,4,Aakanksha,"{'tt2191164': ['0'], 'tt0077783': ['0'], 'tt00..."


In [0]:
train.columns.tolist()

['index', '_id', 'rated']

In [0]:
f=open('FlickscoreData-26oct2018/movies.json','r',encoding = "ISO-8859-1")
users = pd.read_csv('FlickscoreData-26oct2018/users.csv',encoding = "ISO-8859-1", sep='delimiter', header=None)
users.head()
# users.columns  ## Int64Index([0], dtype='int64')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0
0,"_id,languages,job,state,dob,gender"
1,"""11megha89"",""[ """"Hindi"""" ]"",""Student"",""Haryana..."
2,"""2ez4nimzi"",""[ """"Hindi"""" ]"",""Student"",""Delhi"",..."
3,"""9953547227 "",""[ """"Hindi"""" ]"",""Student"",""Delhi..."
4,"""9958221803"",""[ """"Hindi"""" ]"",""Student"",""Delhi""..."


In [0]:
users.columns.tolist()

[0]

In [0]:
users = users[0].str.split(",", expand = True) 


In [0]:
users.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,_id,languages,job,state,dob,gender,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"""11megha89""","""[ """"Hindi"""" ]""","""Student""","""Haryana""","""18-04-1989""","""Female""",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"""2ez4nimzi""","""[ """"Hindi"""" ]""","""Student""","""Delhi""","""16-06-2000""","""Male""",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"""9953547227 ""","""[ """"Hindi"""" ]""","""Student""","""Delhi""","""06-09-1998""","""Male""",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"""9958221803""","""[ """"Hindi"""" ]""","""Student""","""Delhi""","""09-09-1996""","""Male""",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [0]:
userids = []

f = open('FlickscoreData-26oct2018/users.csv', 'r')
user_file = csv.reader(f, delimiter=',')
count = 0
for line in user_file:
    if (count != 0):
        user_id = line[0]
        userids.append(user_id)
    count += 1
f.close()
print(len(userids))
print(userids)

924
['11megha89', '2ez4nimzi', '9953547227 ', '9958221803', 'ABCDEFGHI JKLM', 'ANAND', 'Aakanksha', 'Aakash', 'Aakash22', 'Aashisha', 'Aayushi', 'Abhijeet Mishra ', 'Abhimanyu Tanwar', 'Abhishek', 'Abhishek Gupta', 'Aditipathak', 'Aditya', 'Afsha shah', 'Ahmad', 'Ajay', 'Ajay12', 'Ajay1998', 'Akanksha', 'Akanksha Mishra', 'Akansha', 'Akash', 'Akash28', 'Akash869', 'Akshat', 'Akshay Thakur', 'Alka', 'AlkaGupta', 'Alok Singh', 'Amandeep', 'Amitjain ', 'Anil', 'Anil Kumar Sharma ', 'Anil Marwah', 'Anil kumar', 'Aniruddh', 'AniruddhAggarwal', 'Anita Goel', 'AnjaliUjjainia', 'Anju', 'Ankit', 'Ankur ', 'Ankur Rastogi', 'Ankur Sharma', 'Anmol18', 'Anshul', 'Anshuman Suri', 'Anubhav', 'Anudita', 'Anuprriya', 'Anusha Raj', 'Anvit', 'Any', 'Apoorv', 'Apurupa', 'Arashdeep ', 'Arashdeep Kaur', 'Archana', 'Arjun13022', 'Arnav Kumar', 'Arun', 'Arun Garg', 'Arun Kumar Gupta', 'Arushi_kapoor', 'Asad', 'Ashish', 'Ashish07', 'Ashish16021', 'Ashish_16', 'Ashok', 'Ashok Kumar', 'Ashutosh Brahma', 'Astitv'

In [0]:
itemids = []

f = open('FlickscoreData-26oct2018/movies.csv', 'r')
item_file = csv.reader(f, delimiter=',')
count = 0
for line in item_file:
    if (count != 0):
        item_id = line[0]
        itemids.append(item_id)
    count += 1
f.close()

print(itemids)

['tt5286444', 'tt4434004', 'tt0248126', 'tt0347304', 'tt3043252', 'tt0066763', 'tt2338151', 'tt0169102', 'tt1188996', 'tt0808357', 'tt6040012', 'tt3495026', 'tt4387040', 'tt0075860', 'tt1187043', 'tt4169250', 'tt1010048', 'tt0454876', 'tt3741834', 'tt5571734', 'tt5713232', 'tt0367110', 'tt0292490', 'tt1280558', 'tt4934950', 'tt0457655', 'tt1839596', 'tt2082197', 'tt1562872', 'tt0433416', 'tt0986264', 'tt0367495', 'tt0109117', 'tt0405508', 'tt4430212', 'tt4635372', 'tt2350496', 'tt0109424', 'tt0238936', 'tt4900716', 'tt1954470', 'tt4088588', 'tt2140465', 'tt0449994', 'tt2882328', 'tt0420332', 'tt0242519', 'tt0073707', 'tt0112870', 'tt5310090', 'tt1093370', 'tt3390572', 'tt3767372', 'tt0375611', 'tt1821480', 'tt0240200', 'tt0400234', 'tt0093603', 'tt0172684', 'tt3863552', 'tt2356180', 'tt0374887', 'tt0499375', 'tt2181831', 'tt3447364', 'tt3322420', 'tt5600714', 'tt3848892', 'tt1327035', 'tt0104561', 'tt1230165', 'tt0871510', 'tt1934231', 'tt0488414', 'tt0195231', 'tt0222012', 'tt0466460'

## Preprocessing

In [0]:
f = open('FlickscoreData-26oct2018/user_ids.pkl', 'wb')
pickle.dump(userids, f)
f.close()

f = open('FlickscoreData-26oct2018/item_ids.pkl', 'wb')
pickle.dump(itemids, f)
f.close()

n_users = len(userids)
n_items = len(itemids)

In [0]:
for file_name in ['fold1_train', 'fold2_train', 'fold3_train', 'fold4_train', 'fold5_train', 'fold1_test', 'fold2_test', 'fold3_test', 'fold4_test', 'fold5_test', 'ratings_correctFormat']:

    f = open('FlickscoreData-26oct2018/' + file_name + '.json', 'r')
    print(file_name)
    ratings_file = json.load(f)
    f.close()

    r = np.zeros((n_users, n_items))

    for dic in ratings_file:
        user = userids.index(dic['_id'])
        r_dic = dic['rated']
        for k,v in r_dic.items():
            if (k == 'submit'):
                continue
            item = itemids.index(k)
            rating = int(v[0])
            r[user][item] = rating+2

    f = open('FlickscoreData-26oct2018/user-item-ratings-' + file_name + '.pkl', 'wb')
    pickle.dump(r, f)
    f.close()


fold1_train
fold2_train
fold3_train
fold4_train
fold5_train
fold1_test
fold2_test
fold3_test
fold4_test
fold5_test
ratings_correctFormat


In [0]:
# fs = open('FlickscoreData-26oct2018/user-item-ratings-ratings_correctFormat.pkl', 'rb')
# r = pickle.load(fs)
# fs.close()

# ff = open('FlickscoreData-26oct2018/user_ids.pkl', 'rb')
# userids = pickle.load(ff)
# ff.close()
# n_users = len(userids)
user_vec = np.zeros((924, 25)) 

f = open('FlickscoreData-26oct2018/user_vector.pkl', 'wb')
pickle.dump(user_vec, f)
f.close()
print(f)
print(user_vec)

<_io.BufferedWriter name='FlickscoreData-26oct2018/user_vector.pkl'>
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:

# cwd = os.getcwd() #pwd
for file_name in ['fold1', 'fold2', 'fold3', 'fold4', 'fold5']:

    f = open('FlickscoreData-26oct2018/user-item-ratings-ratings_correctFormat.pkl', 'rb')
    r = pickle.load(f)
    f.close()

    X = []
    y = []
    for u in range(r.shape[0]):
        for i in range(r.shape[1]):
            if (r[u][i] != 0):
                X.append([u, i])
                y.append(int(r[u][i]))

    X = np.array(X)
    y = np.array(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    f = open('FlickscoreData-26oct2018/' + file_name + '_train.txt', 'w')
    for x in range(len(X_train)):
        f.write(str(X_train[x][0]) + "\t" + str(X_train[x][1]) + "\t" + str(y_train[x]) + "\n")
    f.close()

    f = open('FlickscoreData-26oct2018/' + file_name + '_test.txt', 'w')
    for x in range(len(X_test)):
        f.write(str(X_test[x][0]) + "\t" + str(X_test[x][1]) + "\t" + str(y_test[x]) + "\n")
    f.close()


In [0]:
cwd = os.getcwd() #pwd

f = open('FlickscoreData-26oct2018/user_ids.pkl', 'rb')
userids = pickle.load(f)
f.close()

f = open('FlickscoreData-26oct2018/item_ids.pkl', 'rb')
itemids = pickle.load(f)
f.close()
n_users = len(userids)
n_items = len(itemids)

user_vec = np.zeros((n_users, 25)) 
# 18 lang, 2 gender, 5 job, 27 states
user_vec_dic = {'Kannada':0, 'Bengali':1, 'Tamil':2, 'Punjabi':3, 'Gujarati':4, 
                'Manipuri':5, 'Nepali':6, 'Assamese':7, 'Oriya':8, 'Marathi':9, 'Malayalam':10, 
                'Hindi':11, 'Bhojpuri':12, 'Telugu':13, 'Urdu':14, 'Konkani':15, 'Rajasthani':16, 
                'Haryanvi':17, 'Male':18, 'Female':19, 'Student':20, 'Retired':21, 'Others':22, 
                'Self-employed':23, 'Service':24} 

f = open('FlickscoreData-26oct2018/users.csv', 'r')
user_file = csv.reader(f, delimiter=',')
count = 0

for line in user_file:
    if (count != 0):
        user_id = line[0]
        idx = userids.index(user_id)
        words = line[1].split("\"")
        for word in words:
            if (len(word) > 2):
                if '[' in word or ']' in word:
                    continue
                lang = word
                user_vec[idx][user_vec_dic[lang]] = 1        
        if (len(line[2]) > 0):
            job = line[2]
            user_vec[idx][user_vec_dic[job]] = 1
        if (len(line[5]) > 0):
            gender = line[5]
            user_vec[idx][user_vec_dic[gender]] = 1
    count += 1
f.close()

item_vec = np.zeros((n_items, 21))
# 21 genres
item_vec_dic = {'Western':0, 'Thriller':1, 'Biography':2, 
                'Drama':3, 'Romance':4, 'Mystery':5, 'Sci-Fi':6, 'Fantasy':7, 
                'News':8, 'Musical':9, 'Horror':10, 'War':11, 'Adventure':12, 
                'Action':13, 'Crime':14, 'Music':15, 'Family':16, 'Comedy':17, 
                'History':18, 'Animation':19, 'Sport':20}
f = open('FlickscoreData-26oct2018/movies.csv', 'r')
item_file = csv.reader(f, delimiter=',')
count = 0

for line in item_file:
    if (count != 0):
        item_id = line[0]
        idx = itemids.index(item_id)

        words = line[8].split("\"")
        for word in words:
            if (len(word) > 2):
                if '[' in word or ']' in word:
                    continue
                genre = word
                item_vec[idx][item_vec_dic[genre]] = 1
    count += 1
f.close()

f = open('FlickscoreData-26oct2018/user_vector.pkl', 'wb')
pickle.dump(user_vec, f)
f.close()

f = open('FlickscoreData-26oct2018/item_vector.pkl', 'wb')
pickle.dump(item_vec, f)
f.close()

user_item_vec = {}

for u in range(len(user_vec)):
    for i in range(len(item_vec)):
        user_item_vec[(u, i)] = np.append(user_vec[u], item_vec[i]).tolist()

f = open(os.path.join(cwd, r'user_item_vec.pkl'), 'wb')
pickle.dump(user_item_vec, f)
f.close()

In [0]:
def vector_len(v1, v2):
	s1 = 0
	for idx,x in enumerte(v1):
		if(v2[idx] != 0):
			s1+= (x*x)
	s1 = math.sqrt(s1)

	s2 = 0
	for idx,x in enumerte(v2):
		if(v1[idx] != 0):
			s2+= (x*x)
	s2 = math.sqrt(s2)

	return s1 *s2


def MAE_base(actual , predicted_uu , predicted_ii , total_predicted):
	err_uu = 0
	err_ii = 0
	count = 0
	
	for u in range(NUM_OF_USERS):
		for i in range(NUM_OF_ITEMS):
			err_uu += abs(predicted_uu[u][i] - actual[u][i])
			err_ii += abs(predicted_ii[u][i] - actual[u][i])

	return err_uu/total_predicted , err_ii/total_predicted

def MAE(actual , predicted):
	err = 0
	for idx,a in enumerate(actual):
		err += abs(predicted[idx] - a)
	return err/len(actual)

def dot_product(v1, v2):
	    assert len(v1) == len(v2)
	    return sum([x*y for (x,y) in zip(v1, v2)])

def cosine_similarity(v1, v2):
	try:
	    return dot_product(v1, v2) / (vector_len(v1,v2))
	except:
		return 0

In [0]:

def readMoviesFeatures():
	columns = ["movie_id","description","language","released","rating","writer","director","cast","genre","name"]
	movies = pd.read_csv(folder + '/movies.csv', sep=',', encoding='utf-8', usecols=columns)
	movie_id_list = movies.movie_id.values.tolist()
	movie_map = {x:idx for idx,x in enumerate(movie_id_list)}


	languages = movies.language.values.tolist()
	languages_temp = set()
	temp_languages = []
	for l in languages:
		temp_languages.append(str(l[3:-3]).split("\", \""))
		l =set( str(l[3:-3]).split("\", \"") ) 
		for li in l:
			languages_temp.add(li)

	languages = list(languages_temp)
	languages = {x:idx for idx,x in enumerate(languages) }

	ratings = {x:idx for idx,x in enumerate(movies.rating.unique().tolist())}

	directors = movies.director.values.tolist()
	directors_temp = set()
	temp_directors = []
	for l in directors:
		# print(type(l))
		temp_directors.append(str(l[3:-3]).split("\", \""))
		l =set( str(l[3:-3]).split("\", \"") ) 
		for li in l:
			directors_temp.add(li)
	directors = list(directors_temp)
	directors = {x:idx for idx,x in enumerate(directors) }

	genres = movies.genre.values.tolist()
	genres_temp = set()
	temp_genres = []
	for l in genres:
		temp_genres.append(str(l[3:-3]).split("\", \""))
		l =set( str(l[3:-3]).split("\", \"") ) 
		for li in l:
			genres_temp.add(li)
	genres = list(genres_temp)
	genres = {x:idx for idx,x in enumerate(genres) }

	temp_ratings = movies.rating.values.tolist()

	len_genres = len(genres)
	len_directors = len(directors)
	len_languages = len(languages)
	len_ratings = len(ratings)

	length = len_ratings + len_genres + len_directors + len_languages
	movie_features = np.zeros((len(movie_map) , length))

	for idx,x in enumerate(movie_id_list):
		i = 0
		movie_features[idx , i + ratings[temp_ratings[idx]]] = 1

		i += len_ratings
		for g in temp_genres[idx]:
			movie_features[idx , i + genres[g]] = 1
		i += len_genres
		for d in temp_directors[idx]:
			movie_features[idx , i + directors[d]] = 1
		i += len_directors
		for l in temp_languages[idx]:
			movie_features[idx , i + languages[l]] = 1
	return movie_map , movie_features


def test_baseline(fileName  , user_X_user ,item_X_item , user_X_item_train , avg_rating , k = NUM_OF_USERS - 1):
	user_X_item_test = np.zeros((NUM_OF_USERS , NUM_OF_ITEMS))
	total_predicted = 0

	with open(folder+"/" + fileName + "_test.json"  , 'r') as f:
		datas = json.load(f)
		for d in datas:
				u_id = int(users_map[d['_id']])
				for i in d['rated'].keys():
					if(i == "submit"):
						continue
					i_id = int(movies_map[i])
					rating = int(d['rated'][i][0])

				user_X_item_test[u_id][i_id] = rating + 2

	predicted_ratings_uu , predicted_ratings_ii = np.zeros((NUM_OF_USERS , NUM_OF_ITEMS)) , np.zeros((NUM_OF_USERS , NUM_OF_ITEMS))
	for u in range(NUM_OF_USERS):
		for i in range(NUM_OF_ITEMS):

			if(user_X_item_test[u][i] != 0):
				total_predicted += 1

				sorted_u_ind = np.argsort(user_X_user[u][:])[NUM_OF_USERS - k - 1 : NUM_OF_USERS]
				temp = avg_rating[u]
				sim = 0
				for _u in sorted_u_ind:
					if(_u != u):
						sim += abs(user_X_user[u][_u])

				for _u in sorted_u_ind:
					if(u != _u):
						temp += (  (( user_X_user[u][_u]* ( user_X_item_train[_u][i] - avg_rating[_u] ) ) + 1 ) / (sim + k ) )

				predicted_ratings_uu[u][i] = temp


				sorted_i_ind = np.argsort(item_X_item[i][:])[NUM_OF_ITEMS - k - 1 : NUM_OF_ITEMS]
				temp = 0
				sim = 0
				for _i in sorted_i_ind:
					if(_i != i):
						sim += abs(item_X_item[i][_i] )

				for _i in sorted_i_ind:
					if(_i != i):
						temp += ( (( item_X_item[i][_i] * user_X_item_train[u][_i]) + 1 ) / (sim + k ))

				predicted_ratings_ii[u][i] = temp


	return user_X_item_test , predicted_ratings_uu , predicted_ratings_ii , total_predicted

def get_test_data(fileName):
	test_X , test_Y = [] , []
	with open(folder +"/" + fileName + "_test.json" , 'r') as f:
		datas = json.load(f)
		for d in datas:
				u_id = int(users_map[d['_id']])
				for i in d['rated'].keys():
					if(i == "submit"):
						continue
					i_id = int(movies_map[i])
					rating = int(d['rated'][i][0])

					temp = user_features[u_id , :].tolist() + movie_features[i_id, : ].tolist()
					test_X.append(temp)
					test_Y.append(rating)
	return test_X , test_Y

def train_baseline(fileName):
	user_X_item_train = np.zeros((NUM_OF_USERS , NUM_OF_ITEMS))
	avg_rating = {}
	with open(folder +"/" + fileName + "_train.json" , 'r') as f:
		datas = json.load(f)
		for d in datas:
				u_id = int(users_map[d['_id']])
				for i in d['rated'].keys():
					if(i == "submit"):
						continue
					i_id = int(movies_map[i])
					rating = int(d['rated'][i][0])

					if(u_id not in avg_rating):
						avg_rating[u_id] = 0

					avg_rating[u_id] += rating/NUM_OF_ITEMS

					user_X_item_train[u_id][i_id] = rating + 2


	user_X_user, item_X_item = [] , []

	for u in range(NUM_OF_USERS):
		if(u not in avg_rating):
			avg_rating[u] = 0

	for i in range(NUM_OF_USERS):
		temp = []
		v1 = user_X_item_train[i , :]
		for j in range(NUM_OF_USERS):
			if(i == j):
				temp.append(1)
			elif(avg_rating[i] == 0 or avg_rating[j] == 0):
				temp.append(0)
			else:
				v2 = user_X_item_train[j , :]
				temp.append(cosine_similarity(v1, v2))

		user_X_user.append(temp)

	print("user_X_user done")

	for i in range(NUM_OF_ITEMS):
		temp = []
		v1 = user_X_item_train[: , i]
		for j in range(NUM_OF_ITEMS):
			if(i == j):
				temp.append(1)
			else:
				v2 = user_X_item_train[: , j]
				x= cosine_similarity(v1, v2)
				temp.append(x)
				
		item_X_item.append(temp)

	print("item_X_item done")

	j = {"user_X_user" : user_X_user , "item_X_item" : item_X_item}
	json_string = json.dumps(j)
	f = open("./j_" + fileName +".json","w")
	f.write(json_string)
	f.close()

	return np.matrix(user_X_user) , np.matrix(item_X_item) , np.matrix(user_X_item_train) , avg_rating


def readUsersFeatures():
	user_features = {}
	columns = ["_id","languages","job","state","dob","gender"]
	users = pd.read_csv(folder + "/users.csv",sep=',',usecols=columns)
	users_id_list = users._id.values.tolist()
	users_map = { x:idx  for idx,x in enumerate(users_id_list)}


	languages = users.languages.values.tolist()
	languages_temp = set()
	temp_languages = []
	for l in languages:
		try:
			temp_languages.append(str(l[3:-3]).split("\", \""))
			l =set( str(l[3:-3]).split("\", \"") ) 
			for li in l:
				languages_temp.add(li)
		except:
			temp_languages.append([])

	languages = list(languages_temp)
	languages = {x:idx for idx,x in enumerate(languages) }



	jobs = {  x:idx for idx,x in enumerate(users.job.unique().tolist())}

	states = {x : idx for idx,x in enumerate(users.state.unique().tolist())}
	
	genders = {x : idx for idx,x in enumerate(users.gender.unique().tolist())}


	len_languages = len(languages)
	len_jobs = len(jobs)
	len_states = len(states)
	len_genders = len(genders)


	length = len_languages + len_jobs + len_states + len_genders

	temp_jobs = users.job.tolist()
	temp_states = users.state.tolist()
	temp_genders = users.gender.tolist()
	user_features = np.zeros((len(users_id_list) , length))

	for idx,x in enumerate(users_id_list):
		i = 0
		for l in temp_languages[idx]:
			user_features[idx , i + languages[l]] = 1
		i += len_languages

		user_features[idx , i + jobs[temp_jobs[idx]] ] = 1

		i += len_jobs
		user_features[idx , i + states[temp_states[idx]] ] = 1

		i += len_genders

		user_features[idx , i+ genders[temp_genders[idx]]] = 1
	return users_map , user_features


## Algos

In [0]:
### ----------------- SKlearn Algorithms -------------------------------------------------------

folder = "FlickscoreData-26oct2018"
users_map , user_features = readUsersFeatures()
movies_map , movie_features = readMoviesFeatures()

NUM_OF_USERS = len(users_map)
NUM_OF_ITEMS = len(movies_map)

def get_train_data(fileName):
	train_X = []
	train_Y = []

	with open(folder +"/" + fileName + "_train.json" , 'r') as f:
		datas = json.load(f)
		for d in datas:
				u_id = int(users_map[d['_id']])
				for i in d['rated'].keys():
					if(i == "submit"):
						continue
					i_id = int(movies_map[i])
					rating = int(d['rated'][i][0])
					temp = user_features[u_id , :].tolist() + movie_features[i_id, : ].tolist()
					train_X.append(temp)
					train_Y.append(rating)

	return train_X , train_Y

def doAll():
	files = ["fold1" ,"fold2" , "fold3" ,"fold4" , "fold5"]
	K = [10 , 20 , 30 , 40 ,50]

	print("K - Nearest Neighbour \n")
	nmaes , rmses = 0 , 0
	for f in files:
# 		print(f , ":  ")
		train_X , train_Y = get_train_data(f)
		test_X , test_Y = get_test_data(f)
		neigh = KNeighborsClassifier(n_neighbors=7 , algorithm = 'auto' , n_jobs = -1)
		neigh.fit(train_X, train_Y) 
		predicted = (neigh.predict(test_X))
		n = mean_absolute_error(test_Y, predicted)/2
		r = mean_squared_error(test_Y , predicted)
		nmaes += n
		rmses += r
		print(f  ,"-  RMSE:  " , r,"   NMAE: ", n)
	print('\nAverage NMAE : ' , nmaes/5)
	print("Average RMSE : " , rmses/ 5)

	print("\n MLP Classifier\n")
	nmaes , rmses = 0 , 0
	for f in files:
# 		print( f , ":  ")
		train_X , train_Y = get_train_data(f)
		test_X , test_Y = get_test_data(f)
		clf = MLPClassifier(solver='lbfgs', alpha=1e-5,  hidden_layer_sizes=(512,64), random_state=1 , early_stopping = True , max_iter = 500)
		clf.fit(train_X, train_Y)
		predicted = (clf.predict(test_X))
		n = mean_absolute_error(test_Y, predicted)/2
		r = mean_squared_error(test_Y , predicted)
		nmaes += n
		rmses += r
		print(f  ,"-  RMSE:  " , r,"   NMAE: ", n)
	print('\nAverage NMAE : ' , nmaes/5)
	print("Average RMSE : " , rmses/ 5)

	print("\n Linear Discriminant Analysis\n")
  
	nmaes , rmses = 0 , 0
	for f in files:
		train_X , train_Y = get_train_data(f)
		test_X , test_Y = get_test_data(f)
		clf = LDA()
		clf = clf.fit(train_X, train_Y)
		predicted = (clf.predict(test_X))
		n = mean_absolute_error(test_Y, predicted)/2
		r = mean_squared_error(test_Y , predicted)
		nmaes += n
		rmses += r
		print(f  ,"-  RMSE:  " , r,"   NMAE: ", n)
	print('\nAverage NMAE : ' , nmaes/5)
	print("Average RMSE : " , rmses/ 5)
doAll()

K - Nearest Neighbour 

fold1 -  RMSE:   0.4658119658119658    NMAE:  0.1936902966314731
fold2 -  RMSE:   0.423977951309141    NMAE:  0.1731740927882407
fold3 -  RMSE:   0.47938931297709925    NMAE:  0.19323882224645583
fold4 -  RMSE:   0.49866989117291416    NMAE:  0.1966142684401451
fold5 -  RMSE:   0.5055555555555555    NMAE:  0.205

Average NMAE :  0.19234349602126294
Average RMSE :  0.47468093536533507

 MLP Classifier

fold1 -  RMSE:   0.575666163901458    NMAE:  0.22347913524384114
fold2 -  RMSE:   0.5852090032154341    NMAE:  0.23197060174552137
fold3 -  RMSE:   0.5583424209378408    NMAE:  0.21853871319520174
fold4 -  RMSE:   0.6212817412333737    NMAE:  0.23494558645707375
fold5 -  RMSE:   0.6252777777777778    NMAE:  0.23097222222222222

Average NMAE :  0.22798125177277204
Average RMSE :  0.5931554214131769

 Linear Discriminant Analysis



/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


fold1 -  RMSE:   0.4710910005027652    NMAE:  0.1872800402212167
fold2 -  RMSE:   0.42076251722553976    NMAE:  0.17041800643086816
fold3 -  RMSE:   0.468484187568157    NMAE:  0.18822246455834243
fold4 -  RMSE:   0.5192261185006046    NMAE:  0.20423216444981862
fold5 -  RMSE:   0.5088888888888888    NMAE:  0.20277777777777778

Average NMAE :  0.19058609068760474
Average RMSE :  0.4776905425371911


Best results: K Nearest neighbours

Average NMAE :  0.19

Average RMSE :  0.48

# Extra_Stuff


In [0]:
user_id = 0
user_meta = {}
for i in user_info:
    meta_data = ""
    age = int(i[1])
    gender = i[2]
    profession = i[3]
    temp_age=[0]*len(ages_id)
    if gender == 'M':
        temp_age.append(1)
        temp_age.append(0)
    else:
        temp_age.append(0)
        temp_age.append(1)
    temp_prof = [0]*len(profession_id)
    temp_prof[profession_id[profession]]=1
    for j in range(0,len(profession_id)):
        temp_age.append(temp_prof[j])
    user_meta[user_id]=temp_age
    user_id+=1
    for j in range(0,len(ages_id)):
        if ages_id[j]>age:
            temp_age[j-1]=1
            break
           
user_met = df.from_dict(user_meta)
user_met = user_met.T
print(user_met.shape)
user_met.head()

In [0]:
##   RATINGS ------------------

f = open('ml-100k/u.data','r')
# ratings = {}
# ratings = df()
X = []
Y = []
count = 0
for x in f.readlines():
    idx = x.split()
    temp = []
    temp_rat = [0]*5
    temp_rat[int(idx[2])-1] = 1
    temp = sum([user_meta[int(idx[0])-1],item_meta[int(idx[1])]], [])  
    X.append(temp)
    Y.append(temp_rat)
print(len(X),len(X[0]),len(Y),len(Y[0]))
print(X[0])
print(Y[0])

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

X_train = []
y_train = []
X_test = []
y_test = []

linf = []
l2 = []
sigm = []

k_folds = KFold(n_splits = 5,shuffle=False)
for train_index, test_index in k_folds.split(X):
    print("TRAIN:", train_index[0], "          TEST:", test_index[0])
    for i in train_index:
        X_train.append(X[i])
        y_train.append(Y[i])
    for i in test_index:
        X_test.append(X[i])
        y_test.append(Y[i])
    
    #############################
#     20,1500,100
#     5,30,5
# .  train ratings, train features
#     10,200,30
    accuracy = []
    error = []

    for x in range(10,250,10):
        elm_algo = ELM(len(X_train[0]),len(y_train[0]))
        # TO TRY : #rbf_l2 #sigm #tanh
        elm_algo.add_neurons(x, "rbf_linf") 
        elm_algo.train(np.asarray(X_train),np.asarray(y_train))
        pred = elm_algo.predict(np.asarray(X_test))
        Y_predicted = np.argmax(pred, axis=1)
        Y_true = np.argmax(y_test, axis=1)
        
        err = mae(Y_true,Y_predicted)/4
        acc = accuracy_score(Y_true,Y_predicted)
        accuracy.append(acc)
        error.append(err)
    linf.append(sum(accuracy)/len(accuracy))
    print("Accuracy Score linf: ",sum(accuracy)/len(accuracy))
    print("MAE linf: ", sum(error)/len(error))

    accuracy = []
    error = []
    
    for x in range(10,250,10):
        elm_algo = ELM(len(X_train[0]),len(y_train[0]))
        # TO TRY : #rbf_l2 #sigm #tanh
        elm_algo.add_neurons(x, "rbf_l2") 
        elm_algo.train(np.asarray(X_train),np.asarray(y_train))
        pred = elm_algo.predict(np.asarray(X_test))
        Y_predicted = np.argmax(pred, axis=1)
        Y_true = np.argmax(y_test, axis=1)
        
        err = mae(Y_true,Y_predicted)/4
        acc = accuracy_score(Y_true,Y_predicted)
        accuracy.append(acc)
        error.append(err)
    print("Accuracy Score l2: ",sum(accuracy)/len(accuracy))
    print("MAE l2: ", sum(error)/len(error))
    l2.append(sum(accuracy)/len(accuracy))
    
    accuracy = []
    error = []

    for x in range(10,250,10):
        elm_algo = ELM(len(X_train[0]),len(y_train[0]))
        # TO TRY : #rbf_l2 #sigm #tanh
        elm_algo.add_neurons(x, "sigm") 
        elm_algo.train(np.asarray(X_train),np.asarray(y_train))
        pred = elm_algo.predict(np.asarray(X_test))
        Y_predicted = np.argmax(pred, axis=1)
        Y_true = np.argmax(y_test, axis=1)
        
        err = mae(Y_true,Y_predicted)/4
        acc = accuracy_score(Y_true,Y_predicted)
        accuracy.append(acc)
        error.append(err)
    sigm.append(sum(accuracy)/len(accuracy))
    print("Accuracy Score sigm: ",sum(accuracy)/len(accuracy))
    print("MAE sigm: ", sum(error)/len(error))
    
print()
print("Accuracy score ")
print("For rbf_linf: ",sum(linf)/len(linf))
print("For rbf_l2: ",sum(l2)/len(l2))
print("For Sigm: ",sum(sigm)/len(sigm))


Accuracy score :

Fold 1

Accuracy Score linf: 0.345
Accuracy Score l2: 0.345
Accuracy Score sigm: 0.344
Fold 2

Accuracy Score linf: 0.352
Accuracy Score l2: 0.3531
Accuracy Score sigm: 0.352
Fold 3

Accuracy Score linf: 0.3568
Accuracy Score l2: 0.3574
Accuracy Score sigm: 0.356
Fold 4

Accuracy Score linf: 0.356
Accuracy Score l2: 0.357
Accuracy Score sigm: 0.357
Fold 5

Accuracy Score linf: 0.3557
Accuracy Score l2: 0.356
Accuracy Score sigm: 0.356

MAE for each -

- MAE linf: 0.218
- MAE l2: 0.2175
- MAE sigm: 0.217


Final Accuracy scores
For rbf_linf: 0.353
For rbf_l2: 0.35388
For Sigm: 0.35307